# Imports

In [10]:
import numpy as np
import pandas as pd
import geopandas as gpd
import tensorflow as tf
import tensorflow.keras as keras
import libpysal.weights as weights
import pysal.explore as esda

# Set random seed

In [11]:
tf.random.set_seed(42)
np.random.seed(42)

# Load data

In [12]:
raw_df = gpd.read_file("datasets/3_combined/df.gpkg")

# Separate features

In [13]:
features = raw_df.copy()
features["x_coord"] = features["geometry"].centroid.x
features["y_coord"] = features["geometry"].centroid.y
features = features.drop(columns=["geometry", "total"])
labels = features.pop("very_good_health")

# Build model

In [14]:
scores = []

In [15]:
for i in range(10):

    # Build model
    normaliser = keras.layers.Normalization(axis=-1)
    normaliser.adapt(np.array(features))

    model = keras.Sequential([
        normaliser,
        keras.layers.Dense(372, activation="relu"),
        keras.layers.Dense(208, activation="relu"),
        keras.layers.Dense(173, activation="relu"),
        keras.layers.Dense(1)
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.015686), loss="mae"
    )

    # Fit model
    early_stopper = keras.callbacks.EarlyStopping(
        monitor="val_loss", patience=20, restore_best_weights=True
    )

    model.fit(
        features,
        labels,
        batch_size=30,
        epochs=200,
        validation_split=0.2,
        callbacks=[early_stopper],
        verbose=1,
    )

    # Calculate Moran's I and add to scores
    predictions = model.predict(features).flatten()
    residuals = labels - predictions
    features["residuals"] = residuals
    w = weights.KNN.from_dataframe(raw_df, k=8)
    moran = esda.esda.Moran(features["residuals"], w)
    scores.append(moran.I)

Epoch 1/200
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.3570 - val_loss: 0.0376
Epoch 2/200
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0309 - val_loss: 0.0354
Epoch 3/200
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0274 - val_loss: 0.0292
Epoch 4/200
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0266 - val_loss: 0.0381
Epoch 5/200
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0248 - val_loss: 0.0306
Epoch 6/200
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0254 - val_loss: 0.0322
Epoch 7/200
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0234 - val_loss: 0.0297
Epoch 8/200
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0227 - val_loss: 0.0281
Epoch 9/200
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0220 - val_loss: 0.0280
Epoch 10/200
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0216 - val_loss: 0.0304
Epoch 11/200
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0217 - val_loss: 0.0311
Epoch 12/200
125/125 ━━━━━━━━━━━━━━━━━━━━

# Calculate mean and standard deviation

In [21]:
scores_df = pd.DataFrame(scores, columns = ["Moran's I"])
scores_df.loc["Mean"] = np.mean(scores)
scores_df.loc["Std dev"] = np.std(scores)
scores_df["Moran's I"] = scores_df["Moran's I"].round(6)

# Save output

In [22]:
scores_df.to_csv("outputs/data_analyses/moran.csv")